In [38]:
import pandas as pd
import sqlite3
import os
import tkinter as tk
from tkinter import filedialog
import xml.etree.ElementTree as ET
from datetime import datetime

# Ask user to select folder containing the Session XML file
root = tk.Tk()
root.withdraw()  # Hide the root window
selected_folder = filedialog.askdirectory(initialdir='D:/Pro-Sup Test/Data/')
if not selected_folder:
    raise ValueError("No folder was selected.")

# Extract the test_date from the selected folder name
folder_name = os.path.basename(selected_folder)
test_date = folder_name.split('_', 1)[0]  # Extract '2024-08-13' from '2024-08-13_105_Growth Plate_'

# Find the XML file titled "Session" in the selected folder
xml_file_path = ''
for r, dirs, files in os.walk(selected_folder):
    for file in files:
        if file.lower().startswith('session') and file.lower().endswith('.xml'):
            xml_file_path = os.path.join(r, file)
            break
    if xml_file_path:
        break

if not xml_file_path:
    raise FileNotFoundError("No 'Session' XML file found in the selected folder.")

# Parse the XML file
tree = ET.parse(xml_file_path)
root_xml = tree.getroot()

# Extract required fields from XML
name = root_xml.find(".//Name").text
dob = root_xml.find(".//DOB").text
height = root_xml.find(".//Height").text
weight = root_xml.find(".//Weight").text
injury_history = root_xml.find(".//Injury_History").text
season_phase = root_xml.find(".//Season_Phase").text
dynomometer_score = root_xml.find(".//Dynomometer_Score").text
comments = root_xml.find(".//Comments").text

# Calculate age from DOB
dob_date = datetime.strptime(dob, "%Y-%m-%d")
today = datetime.today()
age = today.year - dob_date.year - ((today.month, today.day) < (dob_date.month, dob_date.day))

# Define final columns
final_columns = [
    'name', 'test_date', 'age', 'height', 'weight', 'injury_history', 'season_phase', 'dynomometer_score', 'comments',
    'forearm_rom_0to10', 'forearm_rom_10to20', 'forearm_rom_20to30', 'forearm_rom',
    'tot_rom_0to10', 'tot_rom_10to20', 'tot_rom_20to30', 'tot_rom',
    'num_of_flips_0_10', 'num_of_flips_10_20', 'num_of_flips_20_30', 'num_of_flips',
    'avg_velo_0_10', 'avg_velo_10_20', 'avg_velo_20_30', 'avg_velo'
]

# Create a DataFrame with XML data and NULL for ASCII columns
data = {
    'name': name,
    'test_date': test_date,
    'age': age,
    'height': height,
    'weight': weight,
    'injury_history': injury_history,
    'season_phase': season_phase,
    'dynomometer_score': dynomometer_score,
    'comments': comments,
    'forearm_rom_0to10': None,
    'forearm_rom_10to20': None,
    'forearm_rom_20to30': None,
    'forearm_rom': None,
    'tot_rom_0to10': None,
    'tot_rom_10to20': None,
    'tot_rom_20to30': None,
    'tot_rom': None,
    'num_of_flips_0_10': None,
    'num_of_flips_10_20': None,
    'num_of_flips_20_30': None,
    'num_of_flips': None,
    'avg_velo_0_10': None,
    'avg_velo_10_20': None,
    'avg_velo_20_30': None,
    'avg_velo': None
}

df = pd.DataFrame([data], columns=final_columns)

# Connect to the SQLite database and create table if needed
db_path = 'D:/Pro-Sup Test/pro-sup_data.sqlite'
conn = sqlite3.connect(db_path)

# Ensure table exists
df.head(0).to_sql('pro_sup_data', conn, if_exists='append', index=False)

# Insert the row with XML data and NULL ASCII columns
df.to_sql('pro_sup_data', conn, if_exists='append', index=False)

conn.close()

print(f"XML data inserted for {name} with test date {test_date}.")


XML data inserted for Dienes, Jacob with test date 2024-08-13.


In [39]:
import os
import pandas as pd
import sqlite3

# Extract test_date from folder name
selected_folder = 'D:/Pro-Sup Test/Data/2024-08-13_105_Growth Plate_'
folder_name = os.path.basename(selected_folder)
test_date = folder_name.split('_', 1)[0]

# Path to ASCII file
text_file_path = 'D:/Pro-Sup Test/pro-sup_data.txt'

with open(text_file_path, 'r') as file:
    lines = file.readlines()
    data = [line.strip().split('\t') for line in lines]

# Extract header and data rows
header = data[1]
df_data = [row[1:] for row in data[5:]]
df_ascii = pd.DataFrame(df_data, columns=header)

# Replace hyphens with underscores
df_ascii.columns = [col.replace('-', '_') for col in df_ascii.columns]

row_ascii = df_ascii.iloc[0]

# Extract ASCII data
values = {col: row_ascii.get(col) for col in df_ascii.columns}

# Construct the UPDATE statement
set_clause = ", ".join([f"{col} = ?" for col in values.keys()])
params = list(values.values()) + [name, test_date]

update_sql = f"UPDATE pro_sup_data SET {set_clause} WHERE name = ? AND test_date = ?;"

conn = sqlite3.connect('D:/Pro-Sup Test/pro-sup_data.sqlite')
cursor = conn.cursor()
cursor.execute(update_sql, params)
conn.commit()
conn.close()

print(f"ASCII data updated for {name} on test date {test_date}.")


ASCII data updated for Dienes, Jacob on test date 2024-08-13.
